Below you can find the code to import data from niania.pl website

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

how many pages nianiapl has for chosen location:

In [ ]:
pages = 8

create set to store offer links:

In [ ]:
links = set()

create dictionary from which we will create df later:

In [ ]:
slownik = {'date': [], 'adres': [], 'wynagrodzenie': [], 'etat': [], 'autor': [], 'doswiadczenie': [],
           'wiek_niani': [], 'dodatkowe_wymagania': [], 'jezyk1': [], 'jezyk2':[], 'dzieci': [],
           'liczba_dzieci': []}

getting all links and adding offer links:

In [ ]:
for page in range(1, pages+1):
    page_link = f'''https://niania.pl/praca/warszawa?page={page}'''
    response = requests.get(page_link)
    soup = BeautifulSoup(response.text, 'html.parser')

    tags = soup('a')

    for tag in tags:
        offer_link = tag.get('href',None)  
        if type(offer_link) == str and offer_link.startswith('https://niania.pl/szukam/'):
            links.add(offer_link)

how many offers do we have:

In [ ]:
print(f'number of offers: ', len(links))

iterate through each offer to create connection with every page and get necessary data:

In [ ]:
for link in links:

    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    soup_str = str(soup)

    lista = soup.find('ul', class_="list-style-none summary")
    lista_oczekiwania = soup.find_all('li', class_="pico-tick-before")

    jezyki = soup.find_all('li', class_='pb-3 icon-lang-polish-before')

    try: #first languge
        jezyk1 = jezyki[0].get_text().strip()
    except Exception:
        jezyk1 = ' '

    try: #second languge
        jezyk2 = jezyki[1].get_text().strip()
    except Exception:
        jezyk2 = ' '

    try: #nannys preferred age
        wiek_niani = lista_oczekiwania[3].find("strong").get_text()
    except Exception:
        wiek_niani = ' '

    try: #advertisement author
        autor = lista.select("li:nth-of-type(7)")[0].find("strong").get_text()
    except Exception:
        autor = ' '

    try: #nannys exeprience
        doswiadczenie = lista_oczekiwania[2].find("strong").get_text()
    except Exception:
        doswiadczenie = ' '

    try: #date
        data_dodania = soup.find('time').get_text()
    except Exception:
        data_dodania = ' '

    try: #adress
        adres = lista.select("li:nth-of-type(5)")[0].find("span").get_text()
    except Exception:
        adres = ' '

    try: #salary
        wynagrodzenie = lista.select("li:nth-of-type(4)")[0].find("span").get_text()
    except Exception:
        wynagrodzenie = ' '

    etat = lista.select("li:nth-of-type(3)")[0].find_all("span")
    e = [etat.get_text() for etat in etat]
    etat_lista = [e.strip() for e in e] # type of job

    dodatki = [] # extra requirements:

    for element in range(4, len(lista_oczekiwania)):
        dodatek = lista_oczekiwania[element].get_text().splitlines()
        d = ' '.join(dodatek)
        d = d.strip()
        dodatki.append(d)

    dziecko = soup.find_all('li', class_='pb-2')

    liczba_dzieci = len(dziecko) #number of children
    dzieci = []

    if len(dziecko) > 1:
        for element in range(0, len(dziecko)):
            p = dziecko[element-1].get_text()
            pp = p.strip()
            dzieci.append(pp)
    else:
        dd = dziecko[0].get_text()
        ddd = dd.strip()
        dzieci.append(ddd)

    # data gathering:

    lista_naglowkow = ['date', 'adres', 'wynagrodzenie', 'etat', 'autor', 'doswiadczenie', 'wiek_niani', 'dodatkowe_wymagania',
                       'jezyk1', 'jezyk2', 'dzieci', 'liczba_dzieci']

    dane = [data_dodania, adres, wynagrodzenie, etat_lista, autor, doswiadczenie, wiek_niani, dodatki,
            jezyk1, jezyk2, dzieci, liczba_dzieci]

    for value in range(0, len(dane)):
        try:
            slownik[lista_naglowkow[value]].append(dane[value])
        except Exception:
            slownik[lista_naglowkow[value]].append('NaN')

creating df with all data:

In [ ]:
df = pd.DataFrame(slownik)

creating csv from df:

In [ ]:
df.to_csv('niania_pl_final.csv')